In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
from keras import models
from keras import layers
from keras.utils import to_categorical
import os
import sys
import urllib # check
from collections import Counter # check
import html
import nltk 
nltk.download('punkt') # check
nltk.download('perluniprops') # check
from nltk import word_tokenize
import pickle
import random # check
import progressbar
import keras
from keras import models
from keras import layers
from keras import utils

# This tokenizer is nice, but could cause problems.
try:
    from nltk.tokenize.moses import MosesDetokenizer
    detokenizer = MosesDetokenizer()
    use_moses_detokenizer = True
except:
    use_moses_detokenizer = False

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/iskriyanavasileva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/iskriyanavasileva/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!


# Parameters

In [2]:
# Corpus parameters
#corpus_path = 

# Preprocessing parameters
preprocessed_corpus_path = "product_descr_preprocessed.p"
most_common_words_number = 10000

# Training parameters
train_anyway = False
model_path = "product_descr.h5"
#dataset_size = 5000
sequence_length = 30
epochs = 10
batch_size = 128
hidden_size = 1000

# Helpers

In [3]:
def encode_sequence(sequence, vocabulary):
    """ Encodes a sequence of tokens into a sequence of indices. """

    return [vocabulary.index(element) for element in sequence if element in vocabulary]


def decode_indices(indices, vocabulary):
    """ Decodes a sequence of indices and returns a string. """

    decoded_tokens = [vocabulary[index] for index in indices]
    if use_moses_detokenizer  == True:
        return detokenizer.detokenize(decoded_tokens, return_str=True)
    else:
        return " ".join(decoded_tokens)

# The Corpus

In [4]:
home = os.getenv("HOME")
nlp_repo = os.path.join(home, 'git/nlp-product-sentiment-classification')
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

In [5]:
train_csv_path = os.path.join(nlp_repo, 'data/03_processed/Train.csv')
train_descr = pd.read_csv(train_csv_path)

In [57]:
train_descr['Product_Description'].to_numpy()

array(['The Web Designer\x89Ûªs Guide to iOS (and Android) Apps, today @mention 10 a.m! {link} #sxsw',
       'RT @mention Line for iPad 2 is longer today than yesterday. #SXSW  // are you getting in line again today just for fun?',
       'Crazy that Apple is opening a temporary store in Austin tomorrow to handle the rabid #sxsw eye pad too seekers.',
       ...,
       'RT @mention RT @mention Download 20+ free tracks from @mention Music Sampler @mention including @glove! {link} #SXSW',
       "OH at Texas Social Media Awards: 'You don't need to ask your mother anymore. Just Google it.' #sxswi #sxsw",
       '#Google launching a &quot;major&quot; new social network at #sxsw ... Wonder what that can be...'],
      dtype=object)

In [59]:
corpus_prep = train_descr['Product_Description'].tolist()
corpus_prep[:16]

['The Web Designer\x89Ûªs Guide to iOS (and Android) Apps, today @mention 10 a.m! {link} #sxsw',
 'RT @mention Line for iPad 2 is longer today than yesterday. #SXSW  // are you getting in line again today just for fun?',
 'Crazy that Apple is opening a temporary store in Austin tomorrow to handle the rabid #sxsw eye pad too seekers.',
 'The lesson from Google One Pass: In this digital environment, users want to purchase across every platform with one tool. #sxsw #elonsxsw',
 'RT @mention At the panel: &quot;Your mom has an ipad, designing for boomers&quot; #sxsw',
 'RT @mention I think my effing hubby is in line for an #iPad 2. Can someone point him towards the line-up for wife number #2. #sxswi #sxsw',
 '&quot;Android users are not iPhone users. (They use the Options menu, and Contextual menu)&quot; ~@mention #sxsw',
 'Wow! RT@mention We interrupt your regularly scheduled #sxsw geek programming with big news {link}  #google #circles',
 'Google to Launch New Social Network Called Circl

# Preprocessing the Corpus

In [19]:
test = set(corpus_prep)
len(test)

6352

In [56]:
def preprocess_corpus(corpus_prep):
    """
    Preprocesses the corpus
    
    Args: 
    corpus_prep - the text to be pre-processed
    
    """

    if not os.path.exists(preprocessed_corpus_path):
        print("Preprocessing corpus...")

        # Getting the vocabulary
        #1. Tokenizing
        print("Tokenizing...")
        corpus_string = [word_tokenize(description) for description in corpus_prep]
        corpus_tokens = [item.lower() for sublist in corpus_string for item in sublist]
        print("Number of tokens:", len(corpus_tokens))
        print("Building vocabulary...")
        word_counter = Counter()
        word_counter.update(corpus_tokens)
        print("Length of vocabulary before pruning:", len(word_counter))
        
        #2. Derive the vocabulary - 10.000 most used words
        vocabulary = [key for key, value in word_counter.most_common(most_common_words_number)]
        print("Length of vocabulary after pruning:", len(vocabulary))

        #3. Converting to indices
        print("Index-encoding...")
        indices = encode_sequence(corpus_tokens, vocabulary)
        print("Number of indices:", len(indices))

        #4. Saving
        print("Saving file...")
        pickle.dump((indices, vocabulary), open(preprocessed_corpus_path, "wb"))
    else:
        print("Corpus already preprocessed.")

preprocess_corpus(corpus_prep)

Preprocessing corpus...
Tokenizing...
Number of tokens: 155260
Building vocabulary...
Length of vocabulary before pruning: 9009
Length of vocabulary after pruning: 9009
Index-encoding...
Number of indices: 155260
Saving file...


### Continue work on 15.11.2020

I. Continue with manual tokenization & encoding 
1. Clean more the corpus see repo: https://github.com/dipanjanS/practical-machine-learning-with-python/blob/master/bonus%20content/nlp%20proven%20approach/NLP%20Strategy%20I%20-%20Processing%20and%20Understanding%20Text.ipynb

2. after creating the vocabulary, encode each description via encoder function in helpers (apply directly to df column?)

3. create a dataformat as in the IMDB dataset - CONVERT TO TF DATASETS in https://stackoverflow.com/questions/58362316/how-do-i-go-from-pandas-dataframe-to-tensorflow-batchdataset-for-nlp (point III) 

II. Repeat it with Keras Tokenizer
https://stackoverflow.com/questions/58362316/how-do-i-go-from-pandas-dataframe-to-tensorflow-batchdataset-for-nlp
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

III. Create a data format as in IMDB Dataset --> what is imdb_train in NLP sentiment analysis Tristan's Notebook
https://stackoverflow.com/questions/58362316/how-do-i-go-from-pandas-dataframe-to-tensorflow-batchdataset-for-nlp

IV. !!!!! Validation Set !!!!!! + think about k-fold cross validation as it is a small data set